In [ ]:
print("Test Scucessfully!")

Test Scucessfully!


# Load Dataset and Model

In [ ]:
from datasets import load_dataset

# Load Pre-processed Dataset from HuggingFace
dataset_name = "OneFly7/llama2-SST2-double-end-token"
train_dataset = load_dataset(dataset_name, split="train")

Found cached dataset parquet (/home/9130/.cache/huggingface/datasets/OneFly7___parquet/OneFly7--llama2-sst2-fine-tuning-without-system-info-ff57b5730490a513/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


In [ ]:
train_dataset[0]['text']

'<s>[INST] Sentence: hide new secretions from the parental units  \nSentiment: [/INST] negative </s>'

In [ ]:
sub_dataset = train_dataset.shuffle(seed=42).select(range(100))

Loading cached shuffled indices for dataset at /home/9130/.cache/huggingface/datasets/OneFly7___parquet/OneFly7--llama2-sst2-fine-tuning-without-system-info-ff57b5730490a513/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-e3cdad24c8550174.arrow


In [ ]:
sub_dataset[0]

{'label_text': 'positive',
 'text': '<s>[INST] Sentence: klein , charming in comedies like american pie and dead-on in election ,  \nSentiment: [/INST] positive </s>'}

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline

## Version 2-13b for finetuning
base_model_name = "meta-llama/Llama-2-13b-hf"
# Version 2-13b-chat for few-shot inference
# base_model_name = "meta-llama/Llama-2-13b-chat-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

device_map = {"": 0}

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    device_map=device_map,
    trust_remote_code=True,
    use_auth_token=True
)

base_model.config.use_cache = False

# More info: https://github.com/huggingface/transformers/pull/24906
base_model.config.pretraining_tp = 1 

/usr/local/lib/python3.10/site-packages/transformers/modeling_utils.py:2193: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

## Load Peft and tokenizer

In [7]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer, TrainingArguments
from peft import LoraConfig
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

output_dir = "./results"

In [23]:
print(tokenizer.encode("\nSentiment:", add_special_tokens=False))
print(tokenizer.encode(" \nSentiment:", add_special_tokens=False))
print(tokenizer.encode(sub_dataset[0]["text"], add_special_tokens=False))
print(tokenizer.encode(sub_dataset[1]["text"], add_special_tokens=False))
print(tokenizer.encode(sub_dataset[2]["text"], add_special_tokens=False))

[29871, 13, 29903, 296, 2073, 29901]
[259, 13, 29903, 296, 2073, 29901]
[1, 518, 25580, 29962, 28048, 663, 29901, 24152, 1919, 1373, 4056, 297, 419, 287, 583, 763, 7902, 2185, 5036, 322, 7123, 29899, 265, 297, 8271, 1919, 259, 13, 29903, 296, 2073, 29901, 518, 29914, 25580, 29962, 6374, 29871, 2]
[1, 518, 25580, 29962, 28048, 663, 29901, 367, 15774, 1319, 259, 13, 29903, 296, 2073, 29901, 518, 29914, 25580, 29962, 6374, 29871, 2]
[1, 518, 25580, 29962, 28048, 663, 29901, 10752, 1319, 322, 259, 13, 29903, 296, 2073, 29901, 518, 29914, 25580, 29962, 6374, 29871, 2]


In [24]:
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=50,
    gradient_accumulation_steps=1,
    learning_rate=2e-4,
    logging_steps=2,
    # max_steps=200,
    num_train_epochs=10,
    report_to="none",
)

max_seq_length = 128

# instruction_template = "Sentence:"
response_template = " \nSentiment:"
collator = DataCollatorForCompletionOnlyLM(response_template=response_template, tokenizer=tokenizer)

trainer = SFTTrainer(
    model=base_model,
    train_dataset=sub_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_args,
    data_collator=collator,
)

Loading cached processed dataset at /home/9130/.cache/huggingface/datasets/OneFly7___parquet/OneFly7--llama2-sst2-fine-tuning-without-system-info-ff57b5730490a513/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-637a34fe490710ef.arrow


In [25]:
trainer.train()

Step,Training Loss
2,3.667700
4,2.958600
6,2.035600
8,1.056900
10,0.557300
12,0.327300
14,0.183000
16,0.123400
18,0.101100
20,0.090000


TrainOutput(global_step=20, training_loss=1.110097200423479, metrics={'train_runtime': 445.221, 'train_samples_per_second': 2.246, 'train_steps_per_second': 0.045, 'total_flos': 2944383062016000.0, 'train_loss': 1.110097200423479, 'epoch': 10.0})

In [26]:
trainer.save_model("./models/llama-2-13b-hf-SFT-100-10ep-co")